In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# MEMS Spring-Mass-Damper System Simulation\n",
    "\n",
    "**Author:** Silicon Fabrication Handbook  \n",
    "**License:** MIT  \n",
    "**Description:** Interactive simulation of a MEMS accelerometer modeled as a spring-mass-damper system\n",
    "\n",
    "---\n",
    "\n",
    "## Table of Contents\n",
    "\n",
    "1. [Introduction](#introduction)\n",
    "2. [Theory](#theory)\n",
    "3. [Setup and Imports](#setup)\n",
    "4. [MEMS Accelerometer Class](#class)\n",
    "5. [Step Response Analysis](#step-response)\n",
    "6. [Frequency Response (Bode Plot)](#frequency-response)\n",
    "7. [Noise Analysis](#noise-analysis)\n",
    "8. [Transient Response](#transient-response)\n",
    "9. [Design Space Exploration](#design-space)\n",
    "10. [Interactive Parameter Tuning](#interactive)\n",
    "11. [Summary and Conclusions](#summary)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Introduction <a id=\"introduction\"></a>\n",
    "\n",
    "MEMS (Micro-Electro-Mechanical Systems) accelerometers are widely used in:\n",
    "- Smartphones and tablets (screen rotation, step counting)\n",
    "- Automotive safety (airbag deployment, ESC)\n",
    "- Consumer electronics (gaming controllers, wearables)\n",
    "- Industrial applications (vibration monitoring)\n",
    "\n",
    "This notebook simulates a **capacitive MEMS accelerometer** using a **spring-mass-damper** model.\n",
    "\n",
    "### Key Features:\n",
    "- ✅ Time-domain response analysis\n",
    "- ✅ Frequency response (Bode plots)\n",
    "- ✅ Noise analysis (thermal + electronic)\n",
    "- ✅ Capacitive sensing simulation\n",
    "- ✅ Design trade-off exploration\n",
    "- ✅ Interactive parameter tuning"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Theory <a id=\"theory\"></a>\n",
    "\n",
    "### 2.1 Equation of Motion\n",
    "\n",
    "The proof mass motion is governed by:\n",
    "\n",
    "$$\n",
    "m\\ddot{x} + b\\dot{x} + kx = ma(t)\n",
    "$$\n",
    "\n",
    "Where:\n",
    "- $m$ = proof mass [kg]\n",
    "- $b$ = damping coefficient [N·s/m]\n",
    "- $k$ = spring constant [N/m]\n",
    "- $x$ = displacement [m]\n",
    "- $a(t)$ = input acceleration [m/s²]\n",
    "\n",
    "### 2.2 Key Parameters\n",
    "\n",
    "**Natural frequency:**\n",
    "$$\\omega_n = \\sqrt{\\frac{k}{m}} \\quad [rad/s]$$\n",
    "\n",
    "**Damping ratio:**\n",
    "$$\\zeta = \\frac{b}{2\\sqrt{mk}}$$\n",
    "\n",
    "**Quality factor:**\n",
    "$$Q = \\frac{1}{2\\zeta}$$\n",
    "\n",
    "### 2.3 Transfer Function\n",
    "\n",
    "In Laplace domain:\n",
    "$$H(s) = \\frac{X(s)}{A(s)} = \\frac{1}{s^2 + 2\\zeta\\omega_n s + \\omega_n^2}$$\n",
    "\n",
    "### 2.4 Capacitive Sensing\n",
    "\n",
    "Parallel plate capacitance:\n",
    "$$C = \\frac{\\epsilon_0 A}{d_0 - x}$$\n",
    "\n",
    "For small displacement:\n",
    "$$\\Delta C \\approx C_0 \\frac{x}{d_0}$$"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Setup and Imports <a id=\"setup\"></a>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Standard imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy import signal\n",
    "from pathlib import Path\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# For interactive widgets\n",
    "try:\n",
    "    from ipywidgets import interact, FloatSlider, IntSlider, Output\n",
    "    import ipywidgets as widgets\n",
    "    WIDGETS_AVAILABLE = True\n",
    "except ImportError:\n",
    "    print(\"⚠️  ipywidgets not installed. Interactive features disabled.\")\n",
    "    print(\"   Install with: pip install ipywidgets\")\n",
    "    WIDGETS_AVAILABLE = False\n",
    "\n",
    "# Configure plotting\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "plt.rcParams['font.size'] = 11\n",
    "plt.rcParams['lines.linewidth'] = 2\n",
    "\n",
    "# Create output directory\n",
    "OUTPUT_DIR = Path(\"images\")\n",
    "OUTPUT_DIR.mkdir(exist_ok=True)\n",
    "\n",
    "print(\"✓ Setup complete!\")\n",
    "print(f\"  NumPy version: {np.__version__}\")\n",
    "print(f\"  Output directory: {OUTPUT_DIR}\")\n",
    "print(f\"  Interactive widgets: {'Enabled' if WIDGETS_AVAILABLE else 'Disabled'}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. MEMS Accelerometer Class <a id=\"class\"></a>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class MEMSAccelerometer:\n",
    "    \"\"\"\n",
    "    MEMS accelerometer spring-mass-damper model.\n",
    "    \n",
    "    Parameters:\n",
    "        m: Proof mass [kg]\n",
    "        k: Spring constant [N/m]\n",
    "        b: Damping coefficient [N·s/m]\n",
    "        C0: Nominal capacitance [F]\n",
    "        d0: Nominal gap [m]\n",
    "        A: Electrode area [m²]\n",
    "    \"\"\"\n",
    "    \n",
    "    def __init__(self, m=1e-9, k=10, b=1e-6, C0=1e-12, d0=2e-6, A=1e-8):\n",
    "        self.m = m\n",
    "        self.k = k\n",
    "        self.b = b\n",
    "        self.C0 = C0\n",
    "        self.d0 = d0\n",
    "        self.A = A\n",
    "        \n",
    "        # Derived parameters\n",
    "        self.omega_n = np.sqrt(k / m)\n",
    "        self.f_n = self.omega_n / (2 * np.pi)\n",
    "        self.zeta = b / (2 * np.sqrt(m * k))\n",
    "        self.Q = 1 / (2 * self.zeta) if self.zeta > 0 else np.inf\n",
    "        self.sensitivity = 1 / (k / m)\n",
    "    \n",
    "    def __repr__(self):\n",
    "        return f\"\"\"MEMS Accelerometer:\n",
    "  Mass: {self.m*1e9:.2f} ng\n",
    "  Spring constant: {self.k:.2f} N/m\n",
    "  Damping: {self.b*1e6:.3f} µN·s/m\n",
    "  Natural frequency: {self.f_n/1e3:.2f} kHz\n",
    "  Damping ratio ζ: {self.zeta:.3f}\n",
    "  Quality factor Q: {self.Q:.1f}\n",
    "  Sensitivity: {self.sensitivity*1e9:.2f} nm/g\"\"\"\n",
    "    \n",
    "    def transfer_function(self):\n",
    "        \"\"\"Return H(s) = X(s)/A(s)\"\"\"\n",
    "        num = [1]\n",
    "        den = [1, 2*self.zeta*self.omega_n, self.omega_n**2]\n",
    "        return signal.TransferFunction(num, den)\n",
    "    \n",
    "    def step_response(self, acceleration=9.81, t_max=0.01, n_points=1000):\n",
    "        \"\"\"Simulate step response.\"\"\"\n",
    "        sys = self.transfer_function()\n",
    "        t = np.linspace(0, t_max, n_points)\n",
    "        t_step, x = signal.step(sys, T=t)\n",
    "        return t_step, x * acceleration\n",
    "    \n",
    "    def frequency_response(self, f_min=1, f_max=1e6, n_points=1000):\n",
    "        \"\"\"Calculate frequency response.\"\"\"\n",
    "        sys = self.transfer_function()\n",
    "        w = 2 * np.pi * np.logspace(np.log10(f_min), np.log10(f_max), n_points)\n",
    "        w, H = signal.freqs(sys.num, sys.den, worN=w)\n",
    "        return w / (2 * np.pi), np.abs(H)\n",
    "    \n",
    "    def displacement_to_capacitance(self, x):\n",
    "        \"\"\"Convert displacement to capacitance.\"\"\"\n",
    "        epsilon_0 = 8.854e-12\n",
    "        return epsilon_0 * self.A / (self.d0 - x)\n",
    "    \n",
    "    def capacitance_to_voltage(self, C, V_bias=1.0, C_f=1e-12):\n",
    "        \"\"\"Convert capacitance to voltage.\"\"\"\n",
    "        delta_C = C - self.C0\n",
    "        return -V_bias * delta_C / C_f\n",
    "\n",
    "# Create default instance\n",
    "accel = MEMSAccelerometer()\n",
    "print(accel)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Step Response Analysis <a id=\"step-response\"></a>\n",
    "\n",
    "Analyze the response to a **1g constant acceleration** (step input)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simulate step response\n",
    "t, x = accel.step_response(acceleration=9.81, t_max=0.01)\n",
    "x_nm = x * 1e9  # Convert to nanometers\n",
    "\n",
    "# Calculate settling time (2% criterion)\n",
    "steady_state = x_nm[-1]\n",
    "settling_idx = np.where(np.abs(x_nm - steady_state) < 0.02 * steady_state)[0][0]\n",
    "settling_time = t[settling_idx]\n",
    "\n",
    "# Plot\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))\n",
    "\n",
    "# Displacement\n",
    "ax1.plot(t * 1e3, x_nm, 'b-', linewidth=2.5, label='Displacement')\n",
    "ax1.axhline(steady_state, color='r', linestyle='--', linewidth=1.5,\n",
    "            label=f'Steady state: {steady_state:.2f} nm')\n",
    "ax1.axvline(settling_time * 1e3, color='g', linestyle='--', linewidth=1.5,\n",
    "            label=f'Settling time: {settling_time*1e3:.2f} ms')\n",
    "ax1.set_xlabel('Time [ms]', fontsize=12)\n",
    "ax1.set_ylabel('Displacement [nm]', fontsize=12)\n",
    "ax1.set_title('Step Response to 1g Acceleration', fontsize=14, fontweight='bold')\n",
    "ax1.legend(fontsize=10, loc='lower right')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Capacitance change\n",
    "C = accel.displacement_to_capacitance(x)\n",
    "delta_C_fF = (C - accel.C0) * 1e15\n",
    "ax2.plot(t * 1e3, delta_C_fF, 'r-', linewidth=2.5)\n",
    "ax2.set_xlabel('Time [ms]', fontsize=12)\n",
    "ax2.set_ylabel('Capacitance Change [fF]', fontsize=12)\n",
    "ax2.set_title('Capacitive Sensor Response', fontsize=14, fontweight='bold')\n",
    "ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'step_response.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • Steady-state displacement: {steady_state:.2f} nm\")\n",
    "print(f\"  • Settling time (2%): {settling_time*1e3:.2f} ms\")\n",
    "print(f\"  • Peak capacitance change: {np.max(delta_C_fF):.2f} fF\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Frequency Response (Bode Plot) <a id=\"frequency-response\"></a>\n",
    "\n",
    "Analyze how the accelerometer responds to different input frequencies."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate frequency response\n",
    "f, H = accel.frequency_response(f_min=10, f_max=1e6, n_points=1000)\n",
    "\n",
    "# Convert to dB and phase\n",
    "mag_dB = 20 * np.log10(H)\n",
    "phase = np.angle(H * 2 * np.pi * f, deg=True)\n",
    "\n",
    "# Find -3dB bandwidth\n",
    "mag_max = np.max(mag_dB)\n",
    "bw_idx = np.where(mag_dB >= mag_max - 3)[0]\n",
    "bandwidth = f[bw_idx[-1]] if len(bw_idx) > 0 else f[-1]\n",
    "\n",
    "# Plot Bode diagram\n",
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))\n",
    "\n",
    "# Magnitude\n",
    "ax1.semilogx(f, mag_dB, 'b-', linewidth=2.5)\n",
    "ax1.axhline(mag_max - 3, color='r', linestyle='--', linewidth=1.5, label='-3dB line')\n",
    "ax1.axvline(bandwidth, color='g', linestyle='--', linewidth=1.5,\n",
    "            label=f'BW: {bandwidth/1e3:.1f} kHz')\n",
    "ax1.axvline(accel.f_n, color='orange', linestyle=':', linewidth=2,\n",
    "            label=f'$f_n$: {accel.f_n/1e3:.1f} kHz')\n",
    "ax1.set_ylabel('Magnitude [dB]', fontsize=12)\n",
    "ax1.set_title('Frequency Response (Bode Plot)', fontsize=14, fontweight='bold')\n",
    "ax1.legend(fontsize=10)\n",
    "ax1.grid(True, alpha=0.3, which='both')\n",
    "\n",
    "# Phase\n",
    "ax2.semilogx(f, phase, 'r-', linewidth=2.5)\n",
    "ax2.set_xlabel('Frequency [Hz]', fontsize=12)\n",
    "ax2.set_ylabel('Phase [degrees]', fontsize=12)\n",
    "ax2.grid(True, alpha=0.3, which='both')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig(OUTPUT_DIR / 'frequency_response.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(f\"📊 Results:\")\n",
    "print(f\"  • -3dB Bandwidth: {bandwidth/1e3:.1f} kHz\")\n",
    "print(f\"  • Resonant peak: {mag_max:.1f} dB\")\n",
    "print(f\"  • Quality factor: {accel.Q:.1f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Noise Analysis <a id=\"noise-analysis\"></a>\n",
    "\n",
    "Analyze **thermal (Brownian) noise** and **electronic noise** sources."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Frequency range\n",
    "f_noise = np.logspace(0, 6, 1000)  # 1 Hz to 1 MHz\n",
    "\n",
    "# Constants\n",
    "k_B = 1.38e-23  # Boltzmann constant\n",
    "T = 300  # Temperature [K]\n",
    "\n",
    "# Thermal (Brownian) noise\n",
    "S_x_thermal = np.sqrt(4 * k_B * T * accel.b / accel.k**2)\n",
    "x_thermal = S_x_thermal * np.ones_like(f_noise)\n",
    "a_thermal = x_thermal * accel.k / accel.m / 9.81  # Convert to g/√Hz\n",
    "\n",
    "# Electronic noise (charge amplifier)\n",
    "R_f = 1e9  # Feedback resistor [Ω]\n",
    "C_f = 1e-12  # Feedback capacitor [F]\n",
    "v_n_amplifier = np.sqrt(4 * k_B * T * R_f)\n",
    "\n",
    "V_bias = 1.0\n",
    "sens_V_per_g = V_bias * accel.C0 / C_f / accel.d0 * accel.sens